In [1]:
import os

import minsearch
import niquests
import orjson
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm.notebook import tqdm

In [2]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = niquests.get(docs_url)
assert docs_response.content is not None
documents_raw = orjson.loads(docs_response.content)

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)


In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Using MinSearch


In [4]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], keyword_fields=["course"]
)

In [5]:
q = "The course has already started, can I still enroll?"

In [6]:
index.fit(documents)

In [7]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5,
)

In [8]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [9]:
token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

In [10]:
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": q}],
)

response

ChatCompletion(id='chatcmpl-BJ1DJsvDgckzaoDjTh7etCzwNdIUF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The ability to enroll in a course that has already started typically depends on the institution or platform offering the course. Here are some general considerations:\n\n1. **Contact the Instructor or Institution:** Reach out directly to the course instructor or the institution's admissions/registrar's office. They may allow late enrollment, especially if you're only a short time behind and can catch up quickly.\n\n2. **Online Learning Platforms:** Many online courses (e.g., Coursera, Udemy, edX) are self-paced or have rolling enrollment, meaning you can join even after the course has officially started.\n\n3. **Catch-Up Requirements:** Be prepared to review any missed materials on your own. Some institutions may require you to demonstrate that you're able to catch up on assignments or lectures.\n\n4. **Enrollment Policies:** C

In [11]:
response.choices[0].message.content

"The ability to enroll in a course that has already started typically depends on the institution or platform offering the course. Here are some general considerations:\n\n1. **Contact the Instructor or Institution:** Reach out directly to the course instructor or the institution's admissions/registrar's office. They may allow late enrollment, especially if you're only a short time behind and can catch up quickly.\n\n2. **Online Learning Platforms:** Many online courses (e.g., Coursera, Udemy, edX) are self-paced or have rolling enrollment, meaning you can join even after the course has officially started.\n\n3. **Catch-Up Requirements:** Be prepared to review any missed materials on your own. Some institutions may require you to demonstrate that you're able to catch up on assignments or lectures.\n\n4. **Enrollment Policies:** Check the institution's policies on late enrollment. Some schools have specific deadlines and may charge late fees for enrolling after the official start date.\n

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [13]:
context = ""

for doc in results:
    context = (
        context
        + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    )

In [14]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [15]:
prompt = prompt_template.format(question=q, context=context).strip()

In [16]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: The course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fin

In [17]:
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
)

response.choices[0].message.content

"Yes, you can still enroll in the course after it has started. Even if you don't register, you're eligible to submit the homework assignments. However, be mindful of the deadlines for submitting the final projects."

In [18]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""
    context = ""

    for doc in search_results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content

In [21]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
query = "How do I run kafka?"
answer = rag(query)

print(answer)

To run Kafka, you can follow these specific instructions based on your use case:

1. **Java Kafka: Running producer/consumer/kstreams/etc. in terminal**  
   In the project directory, execute the following command:  
   ```
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```

2. **Python Kafka: If using Python scripts**  
   - Create and activate a virtual environment:  
     ```
     python -m venv env
     source env/bin/activate
     pip install -r ../requirements.txt
     ```
     (Use `env/Scripts/activate` on Windows instead of `env/bin/activate`).
   - Run the necessary Python files in this environment.
   - Ensure all Docker images are up and running prior to executing your Python script.

If this doesn't match your intent or setup, please provide more details.


## Using ElasticSearch


In [23]:
es_client = Elasticsearch("http://localhost:9200")

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [26]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields",
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp",
                    }
                },
            }
        },
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [27]:
result_docs = elastic_search(q)
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [28]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
query = "How do I run kafka?"
answer = rag(query)

print(answer)

To run Kafka:

If you are using Java Kafka and need to run the producer/consumer/kstreams/etc from the terminal, navigate to the project directory and execute the following command:

```bash
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

Make sure to replace `<jar_name>` with the appropriate JAR file name from your build.

If you are facing issues running Python Kafka files, consider creating a virtual environment, installing the required dependencies using `requirements.txt`, and running the files within that environment. See the detailed steps provided in the CONTEXT for managing virtual environments.

If your question pertains to another Kafka use case that is not mentioned here, the CONTEXT does not have additional details. 


